In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
import datetime
import re

In [2]:
#DEFINE FUNÇÕES PRINCIPAIS PARA ACESSO DO BANCO DE DADOS COM A BASE INICIAL

conn = sqlite3.connect('DIST_DB_ANEEL.db')
cursor = conn.cursor()
conn_bi = sqlite3.connect('BD_POWER_BI_FINAL.db')
cursor_bi = conn_bi.cursor()

In [3]:
#PREPARA TABELA COM INFORMAÇÕES UTEIS PARA ANALISE

tableName1 = "DIST_QUALIDADE_ATENDIMENTO_COMERCIAL"
tableName2 = "DIST_IND_DOMINIOS"
columnsNecessary = f"{tableName1}.SigAgente,{tableName1}.AnoIndice,{tableName1}.NumPeriodoIndice,{tableName1}.VlrIndiceEnviado,{tableName2}.DscIndicador "

query = f"""
        SELECT {columnsNecessary}
        FROM {tableName1}
        JOIN {tableName2} ON {tableName1}.SigIndicador = {tableName2}.SigIndicador
        WHERE SigAgente = "ENEL CE"

        """
df = pd.read_sql_query(query,conn)


Artigos + descrição para as filtragens da base de qualidade

- Quant. Prazos Viol. de vistoria de unidade consumidora de área urbana (art. 30)
- Quantidade de religação por suspensão indevida do fornec. (art. 176)
- Quant. Prazos Viol. de Conclusão das obras de até 1 km na rede prim. (art. 34)
- Quant. Prazos Viol. de religação de emergência em área urbana (art. 176)
- Quant. Prazos Viol. de substituição do medidor com defeito (art. 115)
- Quant. Prazos Viol. de religação em área rural (art. 176)
- Quant. Prazos Viol. de ressarc. por meio do pgto, conserto ou subst. (art. 208)
- Quant. Prazos Viol. de ligação de consumidor do grupo B em área rural (art. 31)
- Quant. Prazos Viol. vistoria de unidade consumidora de área rural (art. 30)
- Quant. Prazos Viol. de ligação de unidade consumidora do grupo A (art. 31)
- Quant. Prazos Viol. de religação de emergência em área rural (art. 176)
- Quant. Prazos Viol. de verif. de equip. por ressarc. de dano elétrico (art. 206)
- Quant. Prazos Viol. de Conclusão das obras na rede tensão sec. (art. 34)
- Quant. Prazos Viol. de religação por suspensão indevida do fornec. (art. 176)
- Quant. Prazos Viol. de religação em área urbana (art. 176)
- Quant. Prazos Viol. de reclamação de cobrança ou devol. de diferenças (art. 133)
- Quant. Prazos Viol. de solicitações de aferição dos medidores (art. 137)
- Quant. Prazos Viol. de Início das Obras (art. 34)
- Quant. Prazos Viol. de solução de reclamação com visita técnica (art. 197)
- Quant. Prazos Viol. de ligação de consumidor do grupo B em área urbana (art. 31)
- Quant. Prazos Viol. de ligação de consumidor do grupo B em área rural (art. 31)


\n

- Prazo médio de Conclusão das obras na rede tensão sec. (art. 34)
- Prazo Médio de religação de emergência em área urbana (art. 176)
- Prazo Médio de ligação de consumidor do grupo B em área urbana (art. 31)
- Prazo Médio de religação de emergência em área rural (art. 176)
- Prazo Médio de Início das Obras (art. 34)
- Prazo Médio de vistoria de unidade consumidora de área rural (art. 30)
- Prazo Médio de religação por suspensão indevida do fornec. (art. 176)
- Prazo médio de vistoria de unidade consumidora de área urbana (art. 30)
- Prazo Médio de substituição do medidor com defeito (art. 115)
- Prazo Médio de ressarc. por meio do pgto, conserto ou subst. (art. 208)
- Prazo Médio de religação em área urbana (art. 176)
- Prazo Médio de religação em área rural (art. 176)
- Prazo Médio de solicitações de aferição dos medidores (art. 137)
- Prazo Médio de solução de reclamação com visita técnica (art. 197)
- Prazo médio de Conclusão das obras de até 1 km na rede  prim. (art. 34)
- Prazo Médio de ligação de consumidor do grupo B em área rural
- Prazo Médio de ligação de unidade consumidora do grupo A (art. 31)

In [14]:
#Dataframe resultado da query
df_temp = df.copy()

df_temp = set(df_temp['DscIndicador'])

for e in df_temp:
    if "Prazo Médio de ligação de consumidor do grupo B em área rural" in e:
        print(e)

Prazo Médio de ligação de consumidor do grupo B em área rural (art. 31)


In [23]:
#Dataframe resultado da query
df_analise = df.copy()
df_analise['AnoIndice'] = df_analise['AnoIndice'].astype(str)
df_analise['NumPeriodoIndice'] = df_analise['NumPeriodoIndice'].astype(str)
df_analise['dateTimeRef'] = df_analise['AnoIndice'] + "-" + df_analise['NumPeriodoIndice'] + "-" + "01"
df_analise['dateTimeRef'] = pd.to_datetime(df_analise['dateTimeRef'])
df_analise = df_analise.drop(columns=['AnoIndice','NumPeriodoIndice'])
df_analise['VlrIndiceEnviado'] = df_analise['VlrIndiceEnviado'].str.replace(',','.').astype(float)

#Listas com os tópicos analisados

quantityElements = [
    "Quant. Prazos Viol. de vistoria de unidade consumidora de área urbana (art. 30)",
    "Quantidade de religação por suspensão indevida do fornec. (art. 176)",
    "Quant. Prazos Viol. de Conclusão das obras de até 1 km na rede prim. (art. 34)",
    "Quant. Prazos Viol. de religação de emergência em área urbana (art. 176)",
    "Quant. Prazos Viol. de substituição do medidor com defeito (art. 115)",
    "Quant. Prazos Viol. de religação em área rural (art. 176)",
    "Quant. Prazos Viol. de ressarc. por meio do pgto, conserto ou subst. (art. 208)",
    "Quant. Prazos Viol. de ligação de consumidor do grupo B em área rural (art. 31)",
    "Quant. Prazos Viol. vistoria de unidade consumidora de área rural (art. 30)",
    "Quant. Prazos Viol. de ligação de unidade consumidora do grupo A (art. 31)",
    "Quant. Prazos Viol. de religação de emergência em área rural (art. 176)",
    "Quant. Prazos Viol. de verif. de equip. por ressarc. de dano elétrico (art. 206)",
    "Quant. Prazos Viol. de Conclusão das obras na rede tensão sec. (art. 34)",
    "Quant. Prazos Viol. de religação por suspensão indevida do fornec. (art. 176)",
    "Quant. Prazos Viol. de religação em área urbana (art. 176)",
    "Quant. Prazos Viol. de reclamação de cobrança ou devol. de diferenças (art. 133)",
    "Quant. Prazos Viol. de solicitações de aferição dos medidores (art. 137)",
    "Quant. Prazos Viol. de Início das Obras (art. 34)",
    "Quant. Prazos Viol. de solução de reclamação com visita técnica (art. 197)",
    "Quant. Prazos Viol. de ligação de consumidor do grupo B em área urbana (art. 31)",
    "Quant. Prazos Viol. de ligação de consumidor do grupo B em área rural (art. 31)"
]

termAvarage = [
    "Prazo médio de Conclusão das obras na rede tensão sec. (art. 34)",
    "Prazo Médio de religação de emergência em área urbana (art. 176)",
    "Prazo Médio de ligação de consumidor do grupo B em área urbana (art. 31)",
    "Prazo Médio de religação de emergência em área rural (art. 176)",
    "Prazo Médio de Início das Obras (art. 34)",
    "Prazo Médio de vistoria de unidade consumidora de área rural (art. 30)",
    "Prazo Médio de religação por suspensão indevida do fornec. (art. 176)",
    "Prazo médio de vistoria de unidade consumidora de área urbana (art. 30)",
    "Prazo Médio de substituição do medidor com defeito (art. 115)",
    "Prazo Médio de ressarc. por meio do pgto, conserto ou subst. (art. 208)",
    "Prazo Médio de religação em área urbana (art. 176)",
    "Prazo Médio de religação em área rural (art. 176)",
    "Prazo Médio de solicitações de aferição dos medidores (art. 137)",
    "Prazo Médio de solução de reclamação com visita técnica (art. 197)",
    "Prazo médio de Conclusão das obras de até 1 km na rede prim. (art. 34)",
    "Prazo Médio de ligação de consumidor do grupo B em área rural",
    "Prazo Médio de ligação de unidade consumidora do grupo A (art. 31)",
    "Prazo Médio de ligação de consumidor do grupo B em área rural (art. 31)"
]

#Divisão em dois dataframes com naturezas diferentes:
#1) Dataframe com as quantidades de prazos violados
#2) Dataframe com os prazos médios

quantidadePrazoDataFrame = df_analise[df_analise['DscIndicador'].isin(quantityElements)].copy()
prazoMedioDataFrame = df_analise[df_analise['DscIndicador'].isin(termAvarage)].copy()



In [28]:
#Criando tabelas


# 1)Quantiade Violação prazo para religamento de energia
filterList = ["Quant. Prazos Viol. de ligação de consumidor do grupo B em área urbana (art. 31)",
              "Quant. Prazos Viol. de ligação de consumidor do grupo B em área rural (art. 31)",
              "Quant. Prazos Viol. de ligação de unidade consumidora do grupo A (art. 31)",
]

qtyViolPrazoReligamento = quantidadePrazoDataFrame[quantidadePrazoDataFrame['DscIndicador'].isin(filterList)].copy()

for linha in qtyViolPrazoReligamento.itertuples():
    indicador = linha.DscIndicador
    if indicador == "Quant. Prazos Viol. de ligação de consumidor do grupo B em área rural (art. 31)":
        qtyViolPrazoReligamento.at[linha.Index,"DscIndicador"] = "Quant. Prazos Viol. de ligação de consumidor GRUPO B"
    elif indicador == "Quant. Prazos Viol. de ligação de consumidor do grupo B em área urbana (art. 31)":
        qtyViolPrazoReligamento.at[linha.Index,"DscIndicador"] = "Quant. Prazos Viol. de ligação de consumidor GRUPO B"
    elif indicador == "Quant. Prazos Viol. de ligação de unidade consumidora do grupo A (art. 31)":
        qtyViolPrazoReligamento.at[linha.Index,"DscIndicador"] = "Quant. Prazos Viol. de ligação de consumidor GRUPO A"
    continue

qtyViolPrazoReligamento = pd.DataFrame(qtyViolPrazoReligamento.groupby(['SigAgente','DscIndicador','dateTimeRef'])['VlrIndiceEnviado'].sum())
qtyViolPrazoReligamento.reset_index(inplace=True)


# 2) Quantidade Violação Obras tensao sec
filterList = ["Quant. Prazos Viol. de Conclusão das obras na rede tensão sec. (art. 34)"
]

qtyViolPrazoObraSec = quantidadePrazoDataFrame[quantidadePrazoDataFrame['DscIndicador'].isin(filterList)].copy()
qtyViolPrazoObraSec['DscIndicador'] = "Quant. Prazos Viol. de Conclusão das obras na rede tensão sec."


# 3) Quantidade Violação Relig Suspensao Indevida
filterList = ["Quant. Prazos Viol. de religação por suspensão indevida do fornec. (art. 176)"
]

qtyViolReligSuspIndevida = quantidadePrazoDataFrame[quantidadePrazoDataFrame['DscIndicador'].isin(filterList)].copy()
qtyViolReligSuspIndevida['DscIndicador'] = "Quant. Prazos Viol. de religação por suspensão indevida do fornec."


# 4) Prazo Médio ligação unidade consumidora
filterList = ["Prazo Médio de religação em área urbana (art. 176)",
              "Prazo Médio de religação em área rural (art. 176)",
]

prazoMedioReligacao = prazoMedioDataFrame[prazoMedioDataFrame['DscIndicador'].isin(filterList)].copy()

for linha in prazoMedioReligacao.itertuples():
    indicador = linha.DscIndicador
    if indicador == "Prazo Médio de religação em área urbana (art. 176)":
        prazoMedioReligacao.at[linha.Index,"DscIndicador"] = "Prazo Médio de religação urbana"
    elif indicador == "Prazo Médio de religação em área rural (art. 176)":
        prazoMedioReligacao.at[linha.Index,"DscIndicador"] = "Prazo Médio de religação rural"

# 5) Prazo Médio ligação por grupo 
filterList = [
    "Prazo Médio de ligação de consumidor do grupo B em área rural (art. 31)",
    "Prazo Médio de ligação de unidade consumidora do grupo A (art. 31)",
    "Prazo Médio de ligação de consumidor do grupo B em área urbana (art. 31)",
]

prazoMedioLigacaoPorGrupo = prazoMedioDataFrame[prazoMedioDataFrame['DscIndicador'].isin(filterList)].copy()

for linha in prazoMedioLigacaoPorGrupo.itertuples():
    indicador = linha.DscIndicador
    if indicador == "Prazo Médio de ligação de consumidor do grupo B em área rural (art. 31)":
        prazoMedioLigacaoPorGrupo.at[linha.Index,"DscIndicador"] = "Prazo Médio de Ligação GRUPO B"
    elif indicador == "Prazo Médio de ligação de consumidor do grupo B em área urbana (art. 31)":
        prazoMedioLigacaoPorGrupo.at[linha.Index,"DscIndicador"] = "Prazo Médio de Ligação GRUPO B"
    elif indicador == "Prazo Médio de ligação de unidade consumidora do grupo A (art. 31)":
        prazoMedioLigacaoPorGrupo.at[linha.Index,"DscIndicador"] = "Prazo Médio de Ligação GRUPO A"
    continue

prazoMedioLigacaoPorGrupo = pd.DataFrame(prazoMedioLigacaoPorGrupo.groupby(['SigAgente','DscIndicador','dateTimeRef'])['VlrIndiceEnviado'].sum())
prazoMedioLigacaoPorGrupo.reset_index(inplace=True)

#Faz a media dos prazos medios dos dois tipos de grupo B
for linha in prazoMedioLigacaoPorGrupo.itertuples():
    if linha.DscIndicador == "Prazo Médio de Ligação GRUPO B":
        prazoMedioLigacaoPorGrupo.at[linha.Index,"VlrIndiceEnviado"] = linha.VlrIndiceEnviado/2

# 6) Prazo Médio Solucao Visita Tecnica
filterList = [
    "Prazo Médio de solução de reclamação com visita técnica (art. 197)"
]

prazoMedioSolucaoVisitaTecnica = prazoMedioDataFrame[prazoMedioDataFrame['DscIndicador'].isin(filterList)].copy()
prazoMedioSolucaoVisitaTecnica['DscIndicador'] = "Prazo Médio resolução com visita técnica"


In [29]:
#Sobe tabelas para banco POWER BI

listaTabelas = {
    "qualidadeComercial-qtyViolPrazoReligamento":qtyViolPrazoReligamento,
    "qualidadeComercial-qtyViolPrazoObraSec":qtyViolPrazoObraSec,
    "qualidadeComercial-qtyViolReligSuspIndevida":qtyViolReligSuspIndevida,
    "qualidadeComercial-prazoMedioReligacao":prazoMedioReligacao,
    "qualidadeComercial-prazoMedioLigacaoPorGrupo":prazoMedioLigacaoPorGrupo,
    "qualidadeComercial-prazoMedioSolucaoVisitaTecnica":prazoMedioSolucaoVisitaTecnica
}

for k,v in listaTabelas.items():
    df = v
    df.to_sql(k,conn_bi, if_exists='replace', index=False)